# Helper Files for the Explore Section

### imports

In [1]:
# imports

import pandas as pd
import numpy as np
import regex as re

# Custom Helper Files
from prepare import *

# Stats
from scipy import stats

# Visualize
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

# Remove Limits On Viewing Dataframes
pd.set_option('display.max_columns', None)

In [2]:
df = prepare_atp()

In [3]:
df.head(1)

,tourney_id,tourney_name,surface,draw_size,tourney_level,match_num,score,best_of,round,minutes,player_1,player_2,player_1_age,player_2_age,player_1_entry,player_2_entry,player_1_hand,player_2_hand,player_1_ht,player_2_ht,player_1_id,player_2_id,player_1_ioc,player_2_ioc,player_1_name,player_2_name,player_1_rank,player_2_rank,player_1_rank_points,player_2_rank_points,player_1_seed,player_2_seed,player_1_aces,player_2_aces,player_1_double_faults,player_2_double_faults,player_1_service_points,player_2_service_points,player_1_first_serves_in,player_2_first_serves_in,player_1_first_serve_points_won,player_2_first_serve_points_won,player_1_second_serve_points_won,player_2_second_serve_points_won,player_1_service_game_total,player_2_service_game_total,player_1_break_points_saved,player_2_break_points_saved,player_1_break_points_faced,player_2_break_points_faced,winner,player_1_first_serve_%,player_2_first_serve_%,player_1_first_serve_win_%,player_2_first_serve_win_%,player_1_break_points_won,player_2_break_points_won,player_1_wins,surface_Carpet,surface_Clay,surface_Grass,surface_Hard,tourney_level_A,tourney_level_D,tourney_level_F,tourney_level_G,tourney_level_M,player_1_hand_L,player_1_hand_R,player_2_hand_L,player_2_hand_R,round_ER,round_F,round_QF,round_R128,round_R16,round_R32,round_R64,round_RR,round_SF
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-11,1999-338,Sydney,Hard,32,A,16,7-6(1) 6-1,3,R32,84.0,Lleyton Hewitt,Patrick Rafter,17.878166,26.036961,NaN,NaN,R,R,180.0,185.0,103720,102158,AUS,AUS,Lleyton Hewitt,Patrick Rafter,104.0,4.0,456.0,3315.0,NaN,2.0,2.0,0.0,0.0,5.0,73.0,59.0,51.0,36.0,32.0,24.0,14.0,8.0,10.0,9.0,4.0,3.0,6.0,7.0,Lleyton Hewitt,0.69863,0.610169,0.627451,0.666667,4.0,2.0,True,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0


In [4]:
# Remove Limits On Viewing Dataframes
pd.set_option('display.max_rows', None)

In [5]:
def clean_for_model(df):    
   
    df = df[['tourney_id', 'draw_size', 'winner', 'surface',
       'tourney_level', 'best_of', 'player_1', 'player_2', 'player_1_age',
       'player_2_age', 'player_1_hand',
       'player_2_hand', 'player_1_ht', 'player_2_ht', 'player_1_ioc', 'player_2_ioc', 'player_1_name',
       'player_2_name', 'player_1_rank', 'player_2_rank',
       'player_1_rank_points', 'player_2_rank_points', 'player_1_wins', 'round_ER', 'round_F', 'round_QF', 'round_R128', 'round_R16', 'round_R32', 'round_R64', 'round_RR', 'round_SF', 'player_1_hand_R', 'player_1_hand_L', 
       'tourney_level_A', 'tourney_level_D', 'tourney_level_F', 'tourney_level_G', 'tourney_level_M', 'surface_Carpet', 'surface_Clay',
       'surface_Grass', 'surface_Hard']].copy(0)
    # drop null rows in specific columns
    df = df[df.player_1_ht.notnull()]
    df = df[df.player_2_ht.notnull()]
    
    # winner and loser rank columns
    df['winner_rank'] = np.where(df['winner'] == df['player_1_name'], df['player_1_rank'], df['player_2_rank'])
    df['loser_rank'] = np.where(df['winner'] == df['player_2_name'], df['player_1_rank'], df['player_2_rank'])
    
    # Calculate the difference in stats between player1 and playeer2. Save to new column. 
    df['ht_diff'] = df.player_1_ht - df.player_2_ht
    df['age_diff'] = df.player_1_age - df.player_2_age
    df['rank_diff'] = df.player_1_rank - df.player_2_rank
    df['rank_points_diff'] = df.player_1_rank_points - df.player_2_rank_points
    
    # upset column
    df['no_upset'] = df['winner_rank'] < df['loser_rank']
    
    # year column
    df['year'] = (df['tourney_id'].str[0:4]).astype(int)
    
    return df


In [6]:
df = clean_for_model(df)

In [7]:
df.head()

,tourney_id,draw_size,winner,surface,tourney_level,best_of,player_1,player_2,player_1_age,player_2_age,player_1_hand,player_2_hand,player_1_ht,player_2_ht,player_1_ioc,player_2_ioc,player_1_name,player_2_name,player_1_rank,player_2_rank,player_1_rank_points,player_2_rank_points,player_1_wins,round_ER,round_F,round_QF,round_R128,round_R16,round_R32,round_R64,round_RR,round_SF,player_1_hand_R,player_1_hand_L,tourney_level_A,tourney_level_D,tourney_level_F,tourney_level_G,tourney_level_M,surface_Carpet,surface_Clay,surface_Grass,surface_Hard,winner_rank,loser_rank,ht_diff,age_diff,rank_diff,rank_points_diff,no_upset,year
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-11,1999-338,32,Lleyton Hewitt,Hard,A,3,Lleyton Hewitt,Patrick Rafter,17.878166,26.036961,R,R,180.0,185.0,AUS,AUS,Lleyton Hewitt,Patrick Rafter,104.0,4.0,456.0,3315.0,True,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,104.0,4.0,-5.0,-8.158795,100.0,-2859.0,False,1999
1999-01-11,1999-338,32,Nicolas Kiefer,Hard,A,3,Martin Damm Sr,Nicolas Kiefer,26.444901,21.519507,R,R,188.0,183.0,CZE,GER,Martin Damm Sr,Nicolas Kiefer,75.0,36.0,657.0,1007.0,False,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,36.0,75.0,5.0,4.925394,39.0,-350.0,True,1999
1999-01-11,1999-338,32,Mariano Puerta,Hard,A,3,Jan Siemerink,Mariano Puerta,28.744695,20.312115,L,L,183.0,180.0,NED,ARG,Jan Siemerink,Mariano Puerta,19.0,38.0,1664.0,983.0,False,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,38.0,19.0,3.0,8.432580,-19.0,681.0,False,1999
1999-01-11,1999-338,32,Todd Martin,Hard,A,3,Hicham Arazi,Todd Martin,25.229295,28.511978,L,R,175.0,198.0,MAR,USA,Hicham Arazi,Todd Martin,34.0,16.0,1069.0,1774.0,False,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,16.0,34.0,-23.0,-3.282683,18.0,-705.0,True,1999
1999-01-11,1999-338,32,Carlos Moya,Hard,A,3,Carlos Moya,Thomas Johansson,22.373717,23.802875,R,R,190.0,180.0,ESP,SWE,Carlos Moya,Thomas Johansson,5.0,17.0,3159.0,1761.0,True,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,5.0,17.0,10.0,-1.429158,-12.0,1398.0,True,1999
